# MayoMate

### Set Up Environment
 - Import dependencies
 - Define hyperparameters
 - Set up logging
 - Create directories for outputs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import logging
from mayo import *
from mayo.settings import config as cfg
from mayo.settings import genotype_dict as genotype_dict_master
genotype_dict = genotype_dict_master[cfg["genotype_dict"]]

from mayo.settings import haploid_samples as haploid_samples
from mayo.settings import haploid_samples_ung1d as haploid_samples_ung1d

# Set Reference Chromosomal Intervals for figure drawing and create a DataFrame
from mayo.settings import chromosomes as chromosomes
df_chr = pd.DataFrame(chromosomes, columns=["chromosome", "end_position"])


SNP_NUM = cfg["SNP_NUM"]
CLUSTER_TYPE_ANALYSIS = cfg["CLUSTER_TYPE_ANALYSIS"]
SIG = cfg["SIG"]
CLUST_INTER_MUT_MAX = cfg["CLUST_INTER_MUT_MAX"]

#CLUSTER_TYPE_ANALYSIS = "PMACD" (depricated)
#SIG = 'p001_def' # p05, p01, p005, p001_def, p0005, p0001

logging.basicConfig(
    handlers=[logging.FileHandler(
        filename=f'outputs/logs/main_mayo_{SIG}_{SNP_NUM}snp_{CLUSTER_TYPE_ANALYSIS}.log', #main_mayo_2snps.log, main_mayo_1snp.log
        encoding='utf-8',
        mode='w')],
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:.%(funcName)s: %(message)s')
logger = logging.getLogger(__name__)

diploid = cfg["diploid"]
import BioAid as ba

print(f"Running analysis for {CLUSTER_TYPE_ANALYSIS} clusters ({CLUST_INTER_MUT_MAX} cluster distance) with {SNP_NUM} SNPs and significance level {SIG}")

#cosmetics:
pd.set_option('display.expand_frame_repr', False)

#create directory tree
create_project_directory_tree()

### Import Data and Pre-processing

In [ ]:
#Create reference lookup table for all parental SNPs from parents
parental_origin_db_df = createParentalOriginDF(
    cfg["parents_path"],
    removeA3A_motif=False,
    remove_problematic_positions=True,
    remove_telomeres=True,
    remove_all_repetitive_DNA=True,
    parental_origin_db_df_path_save="outputs/all_parental_SNPs.tsv")

# Load the parental CLC SNPs calls and create a list of Sample objects
samples_parental_SNPs = csvDataFrameImport(
    directory=cfg["parental_SNPs_path"], 
    kind="sample", log=True, 
    format="csv", 
    genotype_dict=genotype_dict)
samples_parental_SNPs.sort()

for i in samples_parental_SNPs:
    i.qualitySNVFilter(diploid=diploid, params={"min_frequency": 30, "min_coverage": 8})

# Perform heterozygosity and ploidy analysis. It modifies objects in samples_parental_SNPs list. Needs to be done before removing heterozygous calls.
heterozygosity_ploidy_output = performHeterozygosityAndPloidyAnalysis(
    samples_parental_SNPs, 
    show_details=False, 
    plot_coverage=False,
    save_plot=False,
    save_high_coverage_SNP_positions=True,
    remove_aneuploid_from_high_coverage_SNP_positions=True,
    parental_origin_db_df=parental_origin_db_df #if None, all SNVs, not just parental, will be shown
    )

for i in samples_parental_SNPs:
    i.removeHeterozygousCalls()

save_payload_to_pickle(samples_parental_SNPs, "outputs/partial_pickles/samples_parental_SNPs_heterozygosity.pkl")
save_payload_to_pickle(parental_origin_db_df, "outputs/partial_pickles/parental_origin_db_df.pkl")


In [ ]:
#reload samples_parental_SNPs from pickle
samples_parental_SNPs = load_payload_from_pickle("outputs/partial_pickles/samples_parental_SNPs_heterozygosity.pkl")
parental_origin_db_df = load_payload_from_pickle("outputs/partial_pickles/parental_origin_db_df.pkl")

In [ ]:
# (optional) draw SNP density curves for parental SNPs

lowDensityMap = loadDensityMap(lowDensity=True, max_distance=500)

data_path = 'outputs/all_parental_SNPs.tsv'
df = pd.read_csv(data_path, sep='\t', header=0)
df = df[df['Parent'] == "parent_1"]

print(df['Chromosome'].value_counts())
print(df['Zygosity'].value_counts())
print(df['Parent'].value_counts())
print(df[df['Zygosity'] != "Homozygous"])

plot_chr_SNP_coverage_by_SNP_distance(
    df, distance_max=5000, distace_step=5,
    inverse=True,
    by_chromosome=False,
    save_path='outputs/SNP_density_poor.tsv',
    fig_path='figures/other/SNP_density_poor.png')

plot_chr_SNP_coverage_by_SNP_distance(
    df, distance_max=5000, distace_step=5,
    inverse=True,
    by_chromosome=True,
    save_path='outputs/SNP_density_poor.tsv',
    fig_path='figures/other/SNP_density_poor_by_chr.png')

plot_chr_SNP_coverage_by_SNP_distance(
    df, distance_max=5000, distace_step=5,
    inverse=False,
    by_chromosome=False,
    save_path='outputs/SNP_density.tsv',
    fig_path='figures/other/SNP_dense.png')

plot_chr_SNP_coverage_by_SNP_distance(
    df, distance_max=5000, distace_step=5,
    inverse=False,
    by_chromosome=True,
    save_path='outputs/SNP_density.tsv',
    fig_path='figures/other/SNP_dense_by_chr.png')

In [ ]:
# (required downstream) create recobination maps for each sample/object, and save them to file

switches_list = createRecombinationMaps(
    samples_parental_SNPs,
    parental_origin_db_df,
    df_chr,
    starts,
    draw_map = False, #set to False for faster processing
    saveMap = False, #set to False for faster processing
    min_SNPs_switch_support = SNP_NUM #=1 -> switch called with 1 supporting SNP, can be set to more however
) 
    
saveSwitchesToFile(switches_list, path=f"outputs/all_switches_{SNP_NUM}snp.tsv")
save_payload_to_pickle(switches_list, f"outputs/partial_pickles/switches_list_{SNP_NUM}snp.pkl")
save_payload_to_pickle(samples_parental_SNPs, f"outputs/partial_pickles/samples_parental_SNPs_{SNP_NUM}snp.pkl")

In [ ]:
#Dealing with SNVs calls (A3A mutations)
from mayo.settings import specific_genotypes_2, specific_genotypes_3, specific_genotypes_4

samples_SNVs = csvDataFrameImport(
    directory=cfg["SNVs_path"],
    kind="switches_overlap",
    log=True,
    format="csv",
    genotype_dict=genotype_dict
    )

check_for_duplicate_sample_names(samples_SNVs)
map_genotype_to_sample(samples_SNVs, genotype_dict)
    
samples_SNVs.sort()

for i in samples_SNVs:
    i.df = removeTelomereSNPs(i.df)
    i.df = removeRepetitiveDNASNPs(i.df, repetitive_DNA_fasta_path=cfg["repetitive_regions_fasta_path"])
    i.qualitySNVFilter(
        stringent=False, 
        diploid=diploid,
        params={"min_coverage": 10, "min_frequency": 20})

for sample in samples_SNVs:
    sample.df["mutation"] = sample.df["Chromosome"].astype(str) + "_" + sample.df["Region"].astype(str) + "_" + sample.df["Reference"] + "_" + sample.df["Allele"]

df_all_mutations = createCombinedSNVsTable(samples_SNVs)
df_all_mutations["genotype"] = df_all_mutations["sample_name"].apply(lambda x: genotype_dict[x] if x in genotype_dict.keys() else "N/A")
print("Before filtering all mutations", df_all_mutations.shape)

#for random spores
filterCommonMutations(
    samples_SNVs, 
    max_duplicates = 10,
    specific_genotypes = [
        'ung1∆',
        "ung1∆NAT",
        "ung1∆ EV",
        'ung1∆ premeiotic non-selected',
        'ung1∆ non-selected',
        'UNG1',
        "spo13∆", 
        "spo13∆spo11∆",
        "spo13∆spo11∆ premeiotic",
        "exo1-nd",
        "pol32∆",
        "exo1-ndpol32∆",
        "sgs1∆C",
        "exo1-ndsgs1∆C"
        ],
    genotypes_dict=genotype_dict, 
    max_genotype_duplicates=3)

#remove non-meiotic (premeiotic) mutations from tetrad/dyad data
filterCommonMutations_sectors_premeiotic(
    samples_SNVs, 
    specific_genotypes_2, 
    specific_genotypes_3, 
    specific_genotypes_4, 
    df_all_mutations, 
    genotype_dict,
    minimum_count_within_genotype=0,
    max_outside_genotype_duplicates=0)

#compute and add JT clusters
compute_and_add_JT_clusters(
    samples_SNVs, 
    pvals_to_test = [0.01, 0.001, 0.0001, 0.00001, 0.000001], 
    max_distance_between_mutations = CLUST_INTER_MUT_MAX, 
    min_cluster_mutations = 3)

showSpecificGenotypeClusters(
    samples_SNVs,
    specific_genotypes_2,
    specific_genotypes_3,
    specific_genotypes_4,
    pval=0.0001)

aggregateSectorsClusters(
    df_clusters=pd.read_csv(f"data/new_clusters/mutation_clusters/mutation_clusters_pval_{SIG}.tsv", sep="\t", header=0, index_col=0),
    specific_genotypes_2=specific_genotypes_2,
    specific_genotypes_3=specific_genotypes_3,
    specific_genotypes_4=specific_genotypes_4,
    genotype_dict=genotype_dict,
    remove_non_concordant=True,
    cluster_outpath_individual=f"outputs/sectors_cluster_table_with_post-meiotic_individuals_with_missing{SIG}.tsv",
    cluster_outpath_totals=f"outputs/sectors_cluster_table_with_post-meiotic_totals_with_missing{SIG}.tsv"
    )

save_payload_to_pickle(samples_SNVs, "outputs/partial_pickles/samples_SNVs_with_post_meiotic.pkl")
save_payload_to_pickle(df_all_mutations, "outputs/partial_pickles/df_all_mutations_with_post_meiotic.pkl")

In [ ]:
# pt 2
samples_SNVs = load_payload_from_pickle("outputs/partial_pickles/samples_SNVs_with_post_meiotic.pkl")
switches_list = load_payload_from_pickle(f"outputs/partial_pickles/switches_list_{SNP_NUM}snp.pkl")
from mayo.settings import specific_genotypes_2, specific_genotypes_3, specific_genotypes_4

#filter out mutations that happened after meiosis
filterCommonMutations_sectors_postmeiotic(
    samples_SNVs, 
    specific_genotypes_2, 
    specific_genotypes_3, 
    specific_genotypes_4, 
    df_all_mutations, 
    genotype_dict,
    minimum_count_within_genotype_2=2,
    minimum_count_within_genotype_3=3,
    minimum_count_within_genotype_4=3,
    max_outside_genotype_duplicates=0)

# since some chr/calls can be diploid/triploid even, we shouldn't remove heterozygous calls or be too stringent with qualitySNVFilter
for i in samples_SNVs:
    i.qualitySNVFilter(
        stringent=False, 
        diploid=diploid,
        params={"min_coverage": 10, "min_frequency": 30})

addSwitchesAndClustersToNormalSamples(samples_SNVs, switches_list, df_clusters_type=CLUSTER_TYPE_ANALYSIS, cluster_significance=SIG)

# remove heterozygous calls in case of aneuploid chromosomes?
for i in samples_SNVs:
    if i.name in get_samples_with_genotype("ung1∆+ung1∆NAT+exo1-nd+pol32∆+ung1∆ non-selected+ung1∆ EV+UNG1+sgs1∆C+exo1-ndsgs1∆C", genotype_dict): #
        len_before = i.df.shape[0]
        mask = ~i.df["Chromosome"].isin(i.aneuploid_chromosomes + i.heterozygous_chromosomes)
        i.df.loc[mask, 'Frequency'] = i.df.loc[mask, 'Frequency'].apply(lambda x: x if x >= 80 else None)
        i.df.dropna(subset=['Frequency'], inplace=True)
        len_after = i.df.shape[0]
        logging.info(f"{i.name}: Removed {len_before - len_after} heterozygous mutation calls from {i.name}. {len_after} calls left.")
        print(f"{i.name}: Removed {len_before - len_after} heterozygous mutation calls from {i.name}. {len_after} calls left.")

#dealing with clusters
compute_and_add_JT_clusters(
    samples_SNVs, 
    pvals_to_test = [0.01, 0.001, 0.0001, 0.00001, 0.000001], 
    max_distance_between_mutations = CLUST_INTER_MUT_MAX, 
    min_cluster_mutations = 3)

showSpecificGenotypeClusters(
    samples_SNVs,
    specific_genotypes_2,
    specific_genotypes_3,
    specific_genotypes_4,
    pval=0.0001)

aggregateSectorsClusters(
    df_clusters=pd.read_csv(f"data/new_clusters/mutation_clusters/mutation_clusters_pval_{SIG}.tsv", sep="\t", header=0, index_col=0),
    specific_genotypes_2=specific_genotypes_2,
    specific_genotypes_3=specific_genotypes_3,
    specific_genotypes_4=specific_genotypes_4,
    genotype_dict=genotype_dict,
    remove_non_concordant=True,
    cluster_outpath_individual=f"outputs/sectors_cluster_table_without_post-meiotic_individuals_with_missing{SIG}.tsv",
    cluster_outpath_totals=f"outputs/sectors_cluster_table_without_post-meiotic_totals_with_missing{SIG}.tsv")
    
save_payload_to_pickle(samples_SNVs, "outputs/partial_pickles/samples_SNVs_without_post_meiotic.pkl")

In [ ]:
parental_origin_db_df = pd.read_csv("outputs/all_parental_SNPs.tsv", sep="\t", header=0, index_col=0)
samples_parental_SNPs = load_payload_from_pickle(f"outputs/partial_pickles/samples_parental_SNPs_{SNP_NUM}snp.pkl")
samples_SNVs = load_payload_from_pickle("outputs/partial_pickles/samples_SNVs_without_post_meiotic.pkl") #samples_SNVs_with_post_meiotic.pkl
switches_list = load_payload_from_pickle(f"outputs/partial_pickles/switches_list_{SNP_NUM}snp.pkl")

In [ ]:
# (optional) finds unreliable SNPs (based on coverage of pooled samples)
findNonRepresentativeSNPs(
    samples_parental_SNPs,
    fig_name_freq_all = "outputs/all_parental_snps_combined_frequency_count_distribution_all60_haploung1d.png",
    fig_name_freq_non_repr = "outputs/all_parental_snps_combined_frequency_count_distribution_first57_haploung1d.png",
    non_representative_SNPs_out_path = "outputs/unreliable_SNPs_freq_count.txt") #output can be used to filter out imbalanced SNPs by adding them to problematic_positions.py

# (optional) find imbalanced SNPs for the dataset (based on frequency of parental alleles in pooled progeny samples)
df_imbalanced_all = find_imbalanced_snps(
    genotype_dict=genotype_dict,
    df_reference_path='outputs/all_parental_SNPs.tsv',
    df_progeny_path='outputs/all_parental_snps_combined.csv',
    genotype_list=[
        "ung1∆",
        "UNG1",
        "ung1∆ non-selected",
        "exo1-nd",
        "pol32∆",
        "exo1-ndpol32∆",
        "ung1∆NAT",
        "ung1∆ EV",
        "exo1-ndsgs1∆C",
        "sgs1∆C",
        ],
    by_genotype=False,
    show=True,
    save=True)

df_imbalanced_all.to_csv("outputs/imbalanced_snps.csv", sep="\t") #save to file #output can be used to filter out imbalanced SNPs by adding them to problematic_positions.py

In [ ]:
#remove rtg samples from both samples_parental_SNPs and samples_SNVs
from mayo.settings import rtg_list
samples_parental_SNPs[:] = [i for i in samples_parental_SNPs if i.name not in rtg_list]
samples_SNVs[:] = [i for i in samples_SNVs if i.name not in rtg_list]
switches_list[:] = [i for i in switches_list if i.name not in rtg_list]

In [ ]:
#SIG = 0.0001 #0.01, 0.001, 0.0001, 0.00001, 0.000001

In [ ]:
# (required downstream) create a cluster table for each sample/object
addSwitchesAndClustersToNormalSamples(samples_SNVs, switches_list, df_clusters_type=CLUSTER_TYPE_ANALYSIS, cluster_significance=SIG)
find_GC_CO_from_switches(samples_SNVs, threshold_GC_max = 5000)
calculate_GC_CO(samples_SNVs)
calculate_scattered_SNPs(samples_SNVs)

#find total amount of ssDNA from clusters (for quick overview)
for p_val in [0.01, 0.001, 0.0001, 0.00001, 0.000001]:
    showTotalssDNAinSamples_SNVs(
        df_total_ssDNA = getTotalssDNAinSamples_SNVs(
            samples_SNVs, 
            cluster_kind = "JT", 
            focus_set = get_samples_with_genotype("ung1∆+ung1∆NAT+exo1-nd+pol32∆+exo1-ndpol32∆+sgs1∆C+exo1-ndsgs1∆C", genotype_dict)),
        p_val=p_val)

heterozygous_chr_counter, aneuploid_chr_counter = find_commonly_heterozygous_chromosomes(samples_parental_SNPs)


In [ ]:
# (optional) save all recombination events to a file
df_all_rec_events = get_all_rec_events_df(
    samples_SNVs, 
    genotype_dict, 
    query_genotypes="ung1∆+ung1∆NAT+UNG1+ung1∆ EV+ung1∆ non-selected+exo1-nd+pol32∆+exo1-ndpol32∆+sgs1∆C+exo1-ndsgs1∆C",
    skip_aneuploid_heterozygous_sample=False,
    skip_aneuploid_heterozygous_chromosome=True)

df_all_rec_events.rename(columns={"Type": "Event"}, inplace=True)
df_all_rec_events.to_csv(f"outputs/all_recombination_events_haploid_chromosomes_{SNP_NUM}snp.csv", index=False, sep="\t", encoding='utf-16')

In [ ]:
#(tests)
from mayo.settings import sectors_wt, sectors_wt_reduced, sectors_spo13, sectors_spo13_reduced, sectors_spo13spo11, sectors_spo13spo11_reduced

wt_counts = []
spo13_counts = []
spo13spo11_counts = []

wt_counts_scattered = []
spo13_counts_scattered = []
spo13spo11_counts_scattered = []


for sample in samples_SNVs:
    if sample.name in sectors_wt_reduced:
        sample_mutations = len(sample.df)
        sample_scattered = len(sample.df_scattered)
        wt_counts.append(sample_mutations)
        wt_counts_scattered.append(sample_scattered)

    elif sample.name in sectors_spo13_reduced:
        sample_mutations = len(sample.df)
        sample_scattered = len(sample.df_scattered)
        spo13_counts.append(sample_mutations)
        spo13_counts_scattered.append(sample_scattered)

    elif sample.name in sectors_spo13spo11_reduced:
        sample_mutations = len(sample.df)
        sample_scattered = len(sample.df_scattered)
        spo13spo11_counts.append(sample_mutations)
        spo13spo11_counts_scattered.append(sample_scattered)


#print average mutation counts
print(f"Average mutations in wt: {np.mean(wt_counts)}")
print(f"Average mutations in spo13: {np.mean(spo13_counts)}")
print(f"Average mutations in spo13spo11: {np.mean(spo13spo11_counts)}")

print(f"Average scattered mutations in wt: {np.mean(wt_counts_scattered)}")
print(f"Average scattered mutations in spo13: {np.mean(spo13_counts_scattered)}")
print(f"Average scattered mutations in spo13spo11: {np.mean(spo13spo11_counts_scattered)}")

#compare by mann whitney u test each set of mutation counts
from scipy.stats import mannwhitneyu
print("Mann Whitney U test results for mutation counts between wt, spo13")
print(mannwhitneyu(wt_counts, spo13_counts))

print("Mann Whitney U test results for mutation counts between wt, spo13spo11")
print(mannwhitneyu(wt_counts, spo13spo11_counts))

print("Mann Whitney U test results for mutation counts between spo13, spo13spo11")
print(mannwhitneyu(spo13_counts, spo13spo11_counts))
        

In [ ]:
# (optional) save SNVs to file
saveClusteredSNVsToFile(samples_SNVs, f"data/new_clusters/clustered_mutations/clustered_SNVs_{CLUSTER_TYPE_ANALYSIS}_{CLUST_INTER_MUT_MAX}_{SIG}.tsv")
saveScatteredSNVsToFile(samples_SNVs, f"data/new_clusters/scattered_mutations/scattered_SNVs_{CLUSTER_TYPE_ANALYSIS}_{CLUST_INTER_MUT_MAX}_{SIG}.tsv")
df_all_mutations = createAllSNVsTable(samples_SNVs, genotypes_keep=None) #genotypes_keep=['ung1∆', "ung1∆NAT", 'ung1∆ non-selected']
df_all_mutations.to_csv(f"outputs/all_mutations.tsv", sep="\t", index=False)

In [ ]:
# save all the data to pickle for cluster association null model simulations
df_clusters=pd.read_csv(f"data/new_clusters/mutation_clusters/mutation_clusters_pval_{SIG}.tsv", sep="\t", header=0, index_col=0)
save_payload_to_pickle([samples_SNVs, switches_list, df_clusters, chromosomes], f"outputs/partial_pickles/saved_state_for_sim_{CLUSTER_TYPE_ANALYSIS}_{CLUST_INTER_MUT_MAX}_{SIG}_{SNP_NUM}snp.pkl")

Past Checkpoint

### Perform Basic Analysis

In [ ]:
output_ratios_list_pvals, clust_mut_vs_total_ssdna_list = calculateClusterStatistics(
    samples_SNVs,
    cluster_kind=CLUSTER_TYPE_ANALYSIS,
    focus_set = get_samples_with_genotype("ung1∆", genotype_dict)  #haploid_samples_ung1d
    )

df_ratios = pd.DataFrame(output_ratios_list_pvals, index=["0.01", "0.001", "0.0001", "0.00001", "0.000001"]).T
samples_count = len(df_ratios)
plot_clust_scattered_ratio_kde(df_ratios, save_path=f"figures/clustered_scattered_ratios_kde_{samples_count}_samples.png", show=True)
plot_clust_scattered_ratio_violin(df_ratios, save_path=f"figures/clustered_scattered_ratios_violin_{samples_count}_samples.png", show=True)

In [ ]:
# !(optional) count the number of A3A signature SNPs in each sample
countA3ASNPs_list(samples_SNVs)

# (optional) time intensive; draw a context plot for combined samples SNVs
genotypes_to_draw = [
    "ung1∆",
    # "ung1∆NAT",
    # "ung1∆ premeiotic non-selected",
    # "UNG1",
    # None, #None will draw all samples
    ]

for genotype in genotypes_to_draw:
    drawContextA3A(
        samples_SNVs,
        show=False,
        save=True,
        focus_set=get_samples_with_genotype(genotype, genotype_dict),
        save_name_suffix=f"{genotype}_only")

In [ ]:
plot_GC_CO_distribution_by_genotype(
    samples_SNVs, 
    query_genotypes = "ung1∆+ung1∆NAT+exo1-nd+pol32∆+exo1-ndpol32∆", #
    save_name = "distribution_haploid",
    show = True
    )

In [ ]:
# (optional) conduct tests on distribution of mutations around recombination events

A3A_mutations_proportion_df = get_proportion_of_A3A_mutations(samples_SNVs)

averages = A3A_mutations_proportion_df.groupby("Genotype")["Total mutations"].mean()
averages = averages[~averages.index.str.contains("sectors")] #if genotype contains "sectors" remove it from averages list
print(averages.head(50))

plot_proportion_A3A_mutations(
    A3A_mutations_proportion_df, 
    genotypes_include = [
        'ung1∆ premeiotic non-selected',
        'UNG1',
        'ung1∆ EV',
        'ung1∆ non-selected', 
        'ung1∆', 
        'ung1∆NAT', 
        'exo1-nd',
        'pol32∆', 
        'exo1-ndpol32∆',
        'sgs1∆C',
        'exo1-ndsgs1∆C',
        'spo13∆', 
        'spo13∆spo11∆', 
        ], 
    fig_size=(16, 6),
    save_path = "figures/A3A_fraction_boxplot_fig2.png")


window_size_norm = 10000

genotype_list = ["ung1∆", "pol32∆", "ung1∆NAT",  "exo1-nd", "exo1-ndpol32∆", "sgs1∆C", "exo1-ndsgs1∆C"]

meta_master_relative_GC_CO_df = get_rec_ev_relative_A3A_SNVs_and_plot(
    samples_SNVs, 
    genotype_dict, 
    genotype_list=genotype_list,
    mutation_type = "all",
    window_size = window_size_norm)

meta_master_relative_GC_CO_df_clustered = get_rec_ev_relative_A3A_SNVs_and_plot(
    samples_SNVs, 
    genotype_dict, 
    genotype_list=genotype_list,
    mutation_type = "clustered",
    window_size = window_size_norm)

meta_master_relative_GC_CO_df_scattered = get_rec_ev_relative_A3A_SNVs_and_plot(
    samples_SNVs, 
    genotype_dict, 
    genotype_list=genotype_list,
    mutation_type = "scattered",
    window_size = window_size_norm)



for index, item in enumerate([
    ["clustered",   meta_master_relative_GC_CO_df_clustered],
    ["scattered", meta_master_relative_GC_CO_df_scattered],
    ["all", meta_master_relative_GC_CO_df]]):

    name = item[0]
    df = item[1]

    print(f"Plotting {name} mutations around recombination events...")

    print(df)

    plot_normalized_rec_A3A(
        df, 
        region = (-8, 8),
        bins = 20,
        poly_order = 1,
        genotypes_include=genotype_list,
        save_path=f"figures/normalized_rec_A3A_8kb_16bins_linear_{name}.png",
        mutation_type=name)

    plot_normalized_rec_A3A(
        df, 
        region = (-12, 12), #in kb
        bins = 24,
        poly_order = 3,
        genotypes_include=genotype_list,
        save_path=f"figures/normalized_rec_A3A_12kb_24bins_3rd_order_{name}.png",
        mutation_type=name)
    
    conduct_wilcoxon_signed_rank_test(df, 8, bins=32)
    conduct_mann_whitney_U_test(df, 8, bins=60)


In [ ]:
# (optional) below's output can be used for chi2 calculation of C->T and G->A balance of mutations around recombination events for different genotypes

combined_meta_master_relative_GC_CO_df = pd.DataFrame()
for mutation_type in ["clustered", "scattered", "all"]:
    print(f"Processing {mutation_type} mutations around recombination events...")
    meta_master_relative_GC_CO_df = get_rec_ev_relative_A3A_SNVs_mod(
        samples_SNVs, 
        genotype_dict, 
        genotype_list = ["ung1∆", "ung1∆NAT", "exo1-nd", "pol32∆", "exo1-ndpol32∆", "sgs1∆C", "exo1-ndsgs1∆C"],
        mutation_type= mutation_type,
        window_size=10000)
    
    meta_master_relative_GC_CO_df["Cluster_pvalue"] = SIG
    meta_master_relative_GC_CO_df["Cluster_inter_mut_max"] = CLUST_INTER_MUT_MAX
    meta_master_relative_GC_CO_df.to_csv(f"outputs/relative_mutations_GC_CO_{mutation_type}.csv", sep="\t", index=False, encoding="utf-16")

    if mutation_type == "all":
        mutation_type = "clustered_and_scattered_(all)"
    meta_master_relative_GC_CO_df["mutation_type_set"] = mutation_type
    combined_meta_master_relative_GC_CO_df = pd.concat([combined_meta_master_relative_GC_CO_df, meta_master_relative_GC_CO_df], ignore_index=True)

combined_meta_master_relative_GC_CO_df.to_csv(f"outputs/relative_mutations_GC_CO_combined.csv", sep="\t", index=False, encoding="utf-16")


In [ ]:
# (optional) plot location of switches (average positions between parental SNPs) on a chosen chromosome
chromosome = 'ref|NC_001135|'
showRecombinationHotspots(
    switches_path="outputs/all_switches_1snp.tsv", 
    chromosome=chromosome,
    genotype_dict=genotype_dict,
    genotype_list=["ung1∆", "ung1∆NAT", "pol32∆", "exo1-nd", "exo1-ndpol32∆"],
    show=True,
    show_most_common=30)

In [ ]:
# (optional) plot location of GC/CO events on chosen all chromosomes
plot_rec_events_jointly(
    df_all_rec_events = get_all_rec_events_df(
        samples_SNVs, 
        genotype_dict, 
        query_genotypes="ung1∆+ung1∆NAT+exo1-nd+pol32∆+exo1-ndpol32∆",
        skip_aneuploid_heterozygous_sample=False,
        skip_aneuploid_heterozygous_chromosome=True),
    event_types = ["GC", "CO"],
    save = True,
    show = True,
    output_path=f"figures/rec_events/rec_events_across_all_chr_{SNP_NUM}.png")

# (optional) plot location of GC/CO events for all chromosomes, each separately
plot_rec_events_for_all_chr(
    df_all_rec_events = get_all_rec_events_df(
        samples_SNVs, 
        genotype_dict, 
        query_genotypes="ung1∆+ung1∆NAT+exo1-nd+pol32∆+exo1-ndpol32∆",
        skip_aneuploid_heterozygous_sample=False,
        skip_aneuploid_heterozygous_chromosome=True),
    SNP_NUM = SNP_NUM,
    event_types = ["GC", "CO"],
    save = True,
    show = True)

In [ ]:
from mayo.settings import sectors_wt, sectors_wt_reduced, sectors_spo13, sectors_spo13_reduced, sectors_spo13spo11, sectors_spo13spo11_reduced

df_master_all, df_master_scattered, df_master_clusters, sample_size = createMasterSNVDataFrames(
    samples_SNVs, 
    sample_names = sectors_spo13_reduced, # sectors_spo13spo11_reduced, sectors_wt_reduced, sectors_spo13_reduced
    #sample_names = get_samples_with_genotype("ung1∆+pol32∆+exo1-nd+exo1-ndpol32∆+sgs1∆C+exo1-ndsgs1∆C", genotype_dict),
    just_heterozygous=False, 
    just_aneuploid=False)

# draw map of all mutations among all samples in df_master_all/scattered/clusters
drawSNPMap(TableSNP(df_master_all, "master_all"), df_chr, starts, showMap=True, saveMap=False, map_type="SNVs_A3A_no_switch")
drawSNPMap(TableSNP(df_master_scattered, "master_scattered"), df_chr, starts, showMap=True, saveMap=False, map_type="SNVs_A3A_no_switch")
drawSNPMap(TableSNP(df_master_clusters, "master_clusters"), df_chr, starts, showMap=True, saveMap=False, map_type="SNVs_A3A_no_switch")

# save df_master_all to pickle
df_master_all.to_pickle("outputs/df_master_all.pkl")
df_master_clusters.to_pickle("outputs/df_master_clusters.pkl")
df_master_scattered.to_pickle("outputs/df_master_scattered.pkl")

Transcription and mutagenesis

In [ ]:
from mayo.settings import non_essential_genes

sample_set = "spo13_tetrad" # spo13spo11_tetrad, wt_tetrad, spo13_tetrad, random_spores
query_positions = ["Start", "End"]

df_mutations = df_master_all.copy()
data_out = []

for query_position in query_positions:

    # number of samples that was used to collect the mutation data
    sample_size_norm = sample_size
    print(f"Using {sample_size_norm} samples")


    ### ORF Coding
    df_feature_mutations_master_coding, df_features = create_mutations_around_features_start_df(
        df_mutations = df_mutations, 
        features_path = "data/orf_coding.fasta", #, "data/orf_coding.fasta", #"data/other_features_genomic.fasta", #,
        query_position = query_position,
        window_size = 1500,
        just_non_essential_genes = non_essential_genes,
        tss_and_tts_from_utrs = True,
        UTR_data_path="data/yeastmine_results_2024-05-24T10-03-55.csv"
        )

    df_nucleotide_freq = get_nucleotide_frequencies_around_feature(df_features, context_flank=1200, query_position=query_position)

    smoothing_method = "rolling" # "rolling", "savgol"
    rolling_window = 100
    region_control = (600, 1200)
    y_lim = None # None (0, 0.000075) (0, 0.001)

    plot_mutations_around_feature(df_feature_mutations_master_coding, title=f"Mutations around Feature {query_position} (protein coding)", window_size=1200)
    plot_mutations_around_feature(df_feature_mutations_master_coding, title=f"Mutations around Feature {query_position} (protein coding)", window_size=500)
    plot_A3A_motifs_relative_to_position(countA3A_motifs_relative_to_position(df_features, context_flank=200, query_position=query_position), suffix=f"orf_coding_200_{query_position}_{sample_set}")
    df_features_mod = countA3A_motifs_relative_to_position(df_features, context_flank=1200, query_position=query_position)
    plot_A3A_motifs_relative_to_position(df_features_mod, suffix=f"orf_coding_1200_{query_position}_{sample_set}")
    df_norm = normalize_mutations_around_feature(df_feature_mutations_master_coding, df_features_mod, feature_range=(-1200, 1200), sample_size_norm=sample_size_norm, feature_count_norm=4410)
    plot_normalized_mutations_around_feature(df_norm, suffix=f"orf_coding_1200_{query_position}_{sample_set}", smoothing_method=smoothing_method, rolling_window=rolling_window, y_lim=y_lim, include_scatter=False)
    plot_normalized_mutations_around_feature(df_norm[(df_norm['Region'] > -500) & (df_norm['Region'] < 500)], suffix=f"orf_coding_500_{query_position}_{sample_set}", smoothing_method=smoothing_method, rolling_window=rolling_window, y_lim=y_lim, include_scatter=False)

    result_list_pk = []
    result_list_pk.append(conduct_mann_u_for_normalized_mutations_around_feature(df_norm, region_test=(-25, 25), region_control=(-1200, -900)))
    result_list_pk.append(conduct_mann_u_for_normalized_mutations_around_feature(df_norm, region_test=(-25, 25), region_control=region_control))
    result_list_pk.append(conduct_mann_u_for_normalized_mutations_around_feature(df_norm, region_test=(-50, 50), region_control=region_control))
    result_list_pk.append(conduct_mann_u_for_normalized_mutations_around_feature(df_norm, region_test=(-100, 100), region_control=region_control))
    result_list_pk.append(conduct_mann_u_for_normalized_mutations_around_feature(df_norm, region_test=(-150, 150), region_control=region_control))

    for result_set in result_list_pk:
        for result in result_set:
            data_out.append({
                "Feature Type": "Protein coding",
                "Spectra": result[0],
                "Position": query_position,
                "Test": result[1],
                "Control": result[2],
                "p-value": result[3],
                "Mean Test": result[4],
                "Mean Control": result[5],
                "Test/Control Ratio": result[4] / result[5]
            })

    ### RNA Coding
    df_feature_mutations_master_rna, df_features_rna = create_mutations_around_features_start_df(
        df_mutations = df_mutations, 
        features_path = "data/rna_coding.fasta",
        query_position = query_position,
        window_size = 1500,
        just_tRNA_genes = True)

    y_lim = None # (0, 0.002) (0, .6)

    plot_mutations_around_feature(df_feature_mutations_master_rna, title=f"Mutations around Feature {query_position} (rna coding)", window_size=1200)
    plot_mutations_around_feature(df_feature_mutations_master_rna, title=f"Mutations around Feature {query_position} (rna coding)", window_size=500)
    plot_A3A_motifs_relative_to_position(countA3A_motifs_relative_to_position(df_features_rna, context_flank=200, query_position=query_position), suffix=f"rna_coding_200_{query_position}_{sample_set}")
    df_features_mod_rna = countA3A_motifs_relative_to_position(df_features_rna, context_flank=1200, query_position=query_position)
    plot_A3A_motifs_relative_to_position(df_features_mod_rna, suffix=f"rna_coding_1200_{query_position}_{sample_set}")
    df_norm = normalize_mutations_around_feature(df_feature_mutations_master_rna, df_features_mod_rna, feature_range=(-1200, 1200), sample_size_norm=sample_size_norm, feature_count_norm=275)
    plot_normalized_mutations_around_feature(df_norm, suffix=f"rna_coding_1200_{query_position}_{sample_set}", smoothing_method=smoothing_method, rolling_window=rolling_window, y_lim=y_lim, include_scatter=False)
    plot_normalized_mutations_around_feature(df_norm[(df_norm['Region'] > -500) & (df_norm['Region'] < 500)], suffix=f"rna_coding_500_{query_position}_{sample_set}", smoothing_method=smoothing_method, rolling_window=rolling_window, y_lim=y_lim, include_scatter=False)

    result_list_trna = []
    
    result_list_trna.append(conduct_mann_u_for_normalized_mutations_around_feature(df_norm, region_test=(0, 100), region_control=region_control))
    result_list_trna.append(conduct_mann_u_for_normalized_mutations_around_feature(df_norm, region_test=(0, 100), region_control=(-1200, -900)))

    for result_set in result_list_trna:
        for result in result_set:
            data_out.append({
                "Feature Type": "tRNA",
                "Spectra": result[0],
                "Position": query_position,
                "Test": result[1],
                "Control": result[2],
                "p-value": result[3],
                "Mean Test": result[4],
                "Mean Control": result[5],
                "Test/Control Ratio": result[4] / result[5]
            })

output_file = f"outputs/mann_u_results_mut_rates_around_features_{sample_set}.csv"

# Create a DataFrame from the collected data
df_out = pd.DataFrame(data_out)
df_out["sample_set"] = sample_set

# Save the DataFrame to a CSV file
df_out.to_csv(output_file, index=False, encoding="utf-16", sep="\t")

# Print the DataFrame
print(df_out)


In [ ]:
#plot the df_nucleotide_freq dataframe where the x-axis is the index and the y-axis is the frequency of each nucleotide
plot_nucleotide_frequencies_around_feature(df_nucleotide_freq, save_name=None, show=True, plot_limits=(-500, 500))


In [ ]:
df_filter = df_feature_mutations_master_coding[(df_feature_mutations_master_coding['Region'] > -75) & (df_feature_mutations_master_coding['Region'] < 75)]

df_filter.Feature.value_counts().to_csv("feature_mutations_counts___sectors.csv")

In [ ]:
#open table S1 from https://doi.org/10.1093/nar/gkac640

df_features_summary = pd.read_csv("df_features_summary.csv", sep=",", header=0)
df_table_S1 = pd.read_csv("data/Table_S1.csv", sep=",", header=0)

keep_cols = [
    'Gene id', 
    'Gene name', 
    'Contig',
    'Start', 
    'End', 
    'Chromosome S288C', 
    'Start S288C', 
    'End S288C',
    'Gene length',

    "WT 0h R1 (normalized and divided by gene length in kb)",
    "WT 0h R2 (normalized and divided by gene length in kb)",
    "WT 0h R3  (normalized and divided by gene length in kb)",

    "WT 2h R1 (normalized and divided by gene length in kb)",
    "WT 2h R2 (normalized and divided by gene length in kb)",
    "WT 2h R3  (normalized and divided by gene length in kb)",

    "WT 3h R1 (normalized and divided by gene length in kb)",
    "WT 3h R2 (normalized and divided by gene length in kb)",
    "WT 3h R3  (normalized and divided by gene length in kb)",

    "WT 5h R1 (normalized and divided by gene length in kb)",
    "WT 5h R2 (normalized and divided by gene length in kb)",
    "WT 5h R3  (normalized and divided by gene length in kb)",

    "WT 6h R1 (normalized and divided by gene length in kb)",
    "WT 6h R2 (normalized and divided by gene length in kb)",
    "WT 6h R3  (normalized and divided by gene length in kb)",

    'WT 7h R1 (normalized and divided by gene length in kb)',
    'WT 7h R2 (normalized and divided by gene length in kb)',
    'WT 7h R3  (normalized and divided by gene length in kb)',

    'WT 9h R1 (normalized and divided by gene length in kb)',
    'WT 9h R2 (normalized and divided by gene length in kb)',
    'WT 9h R3  (normalized and divided by gene length in kb)'
    ]

df_table_S1 = df_table_S1[keep_cols]
df_table_S1 = df_table_S1.rename(columns={
    "Gene id": "feature_name",
    "WT 0h R1 (normalized and divided by gene length in kb)": "WT_0h_R1",
    "WT 0h R2 (normalized and divided by gene length in kb)": "WT_0h_R2",
    "WT 0h R3  (normalized and divided by gene length in kb)":"WT_0h_R3",

    "WT 2h R1 (normalized and divided by gene length in kb)": "WT_2h_R1",
    "WT 2h R2 (normalized and divided by gene length in kb)": "WT_2h_R2",
    "WT 2h R3  (normalized and divided by gene length in kb)":"WT_2h_R3",

    "WT 3h R1 (normalized and divided by gene length in kb)": "WT_3h_R1",
    "WT 3h R2 (normalized and divided by gene length in kb)": "WT_3h_R2",
    "WT 3h R3  (normalized and divided by gene length in kb)":"WT_3h_R3",

    "WT 5h R1 (normalized and divided by gene length in kb)": "WT_5h_R1",
    "WT 5h R2 (normalized and divided by gene length in kb)": "WT_5h_R2",
    "WT 5h R3  (normalized and divided by gene length in kb)":"WT_5h_R3",

    "WT 6h R1 (normalized and divided by gene length in kb)": "WT_6h_R1",
    "WT 6h R2 (normalized and divided by gene length in kb)": "WT_6h_R2",
    "WT 6h R3  (normalized and divided by gene length in kb)":"WT_6h_R3",

    'WT 7h R1 (normalized and divided by gene length in kb)': "WT_7h_R1",
    'WT 7h R2 (normalized and divided by gene length in kb)': "WT_7h_R2",
    'WT 7h R3  (normalized and divided by gene length in kb)':"WT_7h_R3",

    'WT 9h R1 (normalized and divided by gene length in kb)': "WT_9h_R1",
    'WT 9h R2 (normalized and divided by gene length in kb)': "WT_9h_R2",
    'WT 9h R3  (normalized and divided by gene length in kb)':"WT_9h_R3",
    
    })

df_features_summary_merge = df_features_summary.copy()
df_features_summary_merge = df_features_summary_merge[["feature_name", "feature_count"]]

df_features_summary_merge = df_features_summary_merge.merge(df_table_S1, on="feature_name", how="left")

df_mutated_feature = df_features_summary_merge[df_features_summary_merge["feature_count"] > 5].dropna()
df_unaltererd_feature = df_features_summary_merge[df_features_summary_merge["feature_count"] == 0].dropna()

#compare the 'WT 9h R1 (normalized and divided by gene length in kb)' column between the mutated and unaltered features
from scipy.stats import mannwhitneyu
columns = [
    "WT_0h_R1",
    "WT_0h_R2",
    "WT_0h_R3",
    "WT_2h_R1",
    "WT_2h_R2",
    "WT_2h_R3",
    "WT_3h_R1",
    "WT_3h_R2",
    "WT_3h_R3",
    "WT_5h_R1",
    "WT_5h_R2",
    "WT_5h_R3",
    "WT_6h_R1",
    "WT_6h_R2",
    "WT_6h_R3",
    "WT_7h_R1",
    "WT_7h_R2",
    "WT_7h_R3",
    "WT_9h_R1",
    "WT_9h_R2",
    "WT_9h_R3"
    ]

df_heatmap = pd.DataFrame()

for column in columns:
    fold_change = df_mutated_feature[column].mean() / df_unaltererd_feature[column].mean()
    mannwhitneyu_stat, pval = mannwhitneyu(df_mutated_feature[column], df_unaltererd_feature[column])
    genotype, timepoint, replicate = column.split("_")
    new_data = {
        "column": column, 
        "fold_change": fold_change, 
        "pval": pval,
        "genotype": genotype,
        "timepoint": timepoint,
        "replicate": replicate
        }
    df_heatmap = pd.concat([df_heatmap, pd.DataFrame([new_data])]).reset_index(drop=True)
    
    # plt.figure(figsize=(5, 5))
    # df_plot = df_mutated_feature.copy()
    #groupby the feature_count and take the mean of the column
    #df_plot = df_plot.groupby("feature_count")[column].mean().reset_index()
    #sns.regplot(data=df_plot, x="feature_count", y=column, scatter_kws={"s": 20})

#plot the heatmap where y-axis is replicate, x-axis is timepoint and the color is the fold change

df_heatmap = df_heatmap.pivot(index="replicate", columns="timepoint", values="fold_change") #values="pval", "fold_change"
sns.set_context("talk")
plt.figure(figsize=(8, 4))

#make annotations small
from matplotlib.colors import LogNorm, Normalize
#sns.heatmap(df_heatmap, annot=True, cmap=sns.light_palette("seagreen", as_cmap=True, reverse=True), annot_kws={"size":12}, norm=LogNorm())
sns.heatmap(df_heatmap, annot=True, cmap="coolwarm", center=1) 
# plt.title("Fold change in expression levels between genes\n with high mutation rate promoters vs other promoters", fontweight='bold')
#rename the axes
plt.xlabel("Timepoint", fontweight='bold')
plt.ylabel("Replicate", fontweight='bold')
plt.tight_layout()
#save the figure
plt.savefig("fold_change_heatmap.png", dpi=300)
plt.show()

print(df_heatmap)



In [ ]:
# (optional) Draw a map of clusters and switches ticks with distinct colors for A3A C>T and A3A G>A mutations
for i in samples_SNVs:
    drawSNPMap(i, df_chr, starts, saveMap=False, showMap=True, map_type=f"SNVs_A3A_cluster")

In [ ]:
# (required downstream) execute only once!
createDatasetNullHypothesis(switches_list, chromosomes) # this is throwing an error in null hypothesis if executed here, but is needed for overlapClustersWithSwitches

In [ ]:
# (required downstream) find clusters that overlap with recombination events (switches) within a window of 5000 bp, and draw a map of them
for SIG in [0.01, 0.001, 0.00001, 0.000001, 0.0001]:

    df_chr = pd.DataFrame(chromosomes, columns=["chromosome", "end_position"])
    master_counted_clusters_df = overlapClustersWithSwitches(
        samples_SNVs = samples_SNVs,
        window_size = 5000,
        switches_list = switches_list,
        df_chr = df_chr,
        drawMap = False,
        df_clusters_type = CLUSTER_TYPE_ANALYSIS,
        cluster_significance = SIG)

    #save the dataframe to a csv file

    master_counted_clusters_df.to_csv(
        f'outputs/associations/{CLUSTER_TYPE_ANALYSIS}/{SNP_NUM}snp/master_clusters_association_{SIG}_{SNP_NUM}snp_{CLUST_INTER_MUT_MAX}.csv', index=False)

In [ ]:
# (optional) Draw a map of all mutations and switches ticks with distinct colors for A3A C>T and A3A G>A mutations
for i in samples_SNVs:
    drawSNPMap(i, df_chr, starts, saveMap=False, showMap=True, map_type=f"SNVs_clustered")
    break

In [ ]:
sample_names = get_samples_with_genotype("ung1∆+ung1∆NAT+exo1-nd+exo1-ndpol32∆+pol32∆+sgs1∆C+exo1-ndsgs1∆C", genotype_dict)
homozygous_samples = []
for i in samples_SNVs:
    if i.name in sample_names:
        if i.heterozygous == False:
            homozygous_samples.append(i.name)
print(f"there are {len(homozygous_samples)} homozygous samples")
print(homozygous_samples)

df_gaps = getHaplotypeLengthDistributionVsExpected(
    switches_list, 
    filter_set=homozygous_samples,
    inverse=False,
    random_trials=100,
    fraction_increment=20)

In [ ]:
plotHaplotypeLengthDistributionVsExpected(
    df_gaps,
    show=True, 
    figsize=(8, 5),
    yticks=np.arange(0, 0.51, 0.1),
    save_path="figures/haplotype_length_distribution_vs_expected_100_1snp.png")

df_blocks = getHaplotypeBlockLengths(switches_list=switches_list, filter_set=homozygous_samples, random=False, inverse=False)
df_blocks["Distance_kb"] = df_blocks["Distance"]/1000
df_blocks_mod = df_blocks[df_blocks["Distance_kb"] <= 10]

plotHaplotypeLengthDistributionHist(
    df_blocks=df_blocks_mod,
    figsize=(10, 5),
    xticks=np.arange(0, 10, 1),
    show=True,
    save_path=f"figures/haplotype_block_length_distribution_{SNP_NUM}snp_50kb.png")

df = smoothen_derive_col_data(df_gaps, "Fraction_real", window_length = 20, derivative = 1)

plot_col_series(
    df=df, 
    col_name="Fraction_real", 
    x_col="Distance_kb",
    show=True, 
    show_smoothed_derived=True,
    show_original=False,
    title="Rate of change in cumulative fraction of haplotype block lengths",
    save_path=f"figures/haplotype_block_rate_{SNP_NUM}snp_50kb.png",
    label="Rate",
    x_label="Haplotype block length (kb)",
    y_label="Rate of haplotype block fraction gain",
    )

In [ ]:
save_file_name_clusters_null="proximal_clusters_125_1snp_full_combined.csv"
save_path_fig="figures/null_hypothesis/proximal_clusters_10k_haploid_1snp.png"
drawNullHypothesisSimulationResults(save_file_name_clusters_null, show=True, as_percent=True, bootstrap=False, other_clusters=False, save_path=save_path_fig)

In [ ]:
#before starting, save the state of samples_SNVs, switches_list, and chromosomes
payload_path = f"outputs\partial_pickles\samples_parental_SNPs_{CLUSTER_TYPE_ANALYSIS}_{SIG}_{SNP_NUM}snp.pkl"
save_payload_to_pickle([samples_SNVs, switches_list, chromosomes], payload_path)

samples_SNVs, switches_list, chromosomes = load_payload_from_pickle(payload_path)

In [ ]:
#draw cluster_length distribution
from mayo.settings import rtg_list

df_clusters = pd.read_csv("data/new_clusters/mutation_clusters/mutation_clusters_pval_0.0001.tsv", sep="\t")
sample_names = get_samples_with_genotype("ung1∆+ung1∆NAT+ung1∆ non-selected", genotype_dict) #+exo1-nd+pol32∆+exo1-ndpol32∆, ung1∆+ung1∆NAT+ung1∆ non-selected
df_clusters = df_clusters[df_clusters["sample"].isin(sample_names)]
df_clusters["genotype"] = df_clusters["sample"].map(genotype_dict)
df_clusters = df_clusters[~df_clusters["sample"].isin(rtg_list)]

sns.set_style("whitegrid")

#plot distributions of cluster lengths
df_clusters = df_clusters.rename(columns={"genotype": "Genotype"})
#italicize the genotype names
df_clusters["Genotype"] = df_clusters["Genotype"].apply(lambda x: f"${x}$")
sns.set_context("poster")
plt.figure(figsize=(20, 5))
palette = {
    "$ung1∆$": "tab:blue",
    "$ung1∆NAT$": "tab:purple",
    "$ung1∆ non-selected$": "tab:green"
}
sns.histplot(data=df_clusters, x="Length", bins=250, hue="Genotype", palette=palette, fill=True, multiple="stack", alpha=0.85)
plt.xlabel("Cluster length (bp)", fontweight='bold')
plt.ylabel("Number of clusters", fontweight='bold')
plt.title("Distribution of cluster lengths", fontweight='bold')

#save the figure
plt.savefig("figures/cluster_length_distribution.png", dpi=300, bbox_inches='tight')

In [ ]:
def drawSNPMap(
        SampleTable_Object,
        df_chr_lengths: pd.DataFrame,
        chr_starts_df: pd.DataFrame,
        saveMap: bool = True,
        map_type: str = "switches",
        showMap: bool = False,
        save_name_suffix: str = "",
        cluster_kind: str | None = None
        ) -> None:
    '''Draws a SNP map for a given SampleTable object and saves it as a PNG file.

    Args:
        SampleTable_Object (SampleTable): A SampleTable object representing the 
            sample to be plotted.
        df_chr_lengths (pandas.DataFrame): A pandas DataFrame containing the 
            lengths of each chromosome.
        chr_starts_df (pandas.DataFrame): A pandas DataFrame containing the 
            start positions of each chromosome.
        saveMap (bool, optional): A boolean indicating whether to save the map 
            as a PNG file. Defaults to True.
        map_type (str, optional): A string indicating the type of SNP map to 
            draw. Possible values are "switches", "SNPs", "SNVs_A3A_cluster", and "SNVs_A3A_all"
            Defaults to "switches".
        showMap (bool, optional): A boolean indicating whether to show the map 
            in a window. Defaults to False.

    Returns:
        None
    '''

    # style with ticks
    sns.set_style("ticks")
    sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
    
    fig, ax = plt.subplots()
    title = f"{SampleTable_Object.name}" #{round(SampleTable_Object.percent,3)}
    
    df_chr_lengths = renameChrToRoman(df_chr_lengths, "chromosome")
    #use #E6E6E6 for gray color
    df_chr_lengths.plot(kind='barh',legend=False, ax=ax, x="chromosome", y="end_position", fontsize=15, figsize=(20,6), edgecolor="black", linewidth=1, color="#E6E6E6" , zorder=2, title=title, label="Position")

    #To rename legend elements, change plot markers/colors, modify here
    label_dict = {"REF": "Reference Allele"}
    
    if map_type=="switches":
        palette = {"parent_1": "tab:green", "parent_2": "tab:red"}
        markers = {"parent_1": "|", "parent_2": "|"}

        pd_df = SampleTable_Object.output_df.copy()
        pd_df = pd.concat([pd_df, chr_starts_df], ignore_index=True)
        pd_df = renameChrToRoman(pd_df, "Chromosome")
        sns.scatterplot(ax=ax, data=pd_df, x="Region", y="Chromosome", hue="Parent", palette=palette, style="Parent", markers=markers, s=120, alpha=0.5,zorder=3, linewidth=1.5)

    #renamed from SNPs to SNVs
    elif map_type=="SNVs":
        pd_df = SampleTable_Object.df.copy()
        pd_df = pd.concat([pd_df, chr_starts_df], ignore_index=True)
        pd_df = renameChrToRoman(pd_df, "Chromosome")
        sns.scatterplot(ax=ax, data=pd_df, x="Region", y="Chromosome", marker="|", s=120, alpha=0.5,zorder=3, linewidth=1.5)
    
    elif map_type=="SNVs_A3A_no_switch":
        pd_df = SampleTable_Object.df.copy()
        pd_df = pd_df[(((pd_df["Reference"] == "C") & (pd_df["Allele"] == "T")) | ((pd_df["Reference"] == "G") & (pd_df["Allele"] == "A")))]
        pd_df = pd.concat([pd_df, chr_starts_df], ignore_index=True).sort_values(by=["Chromosome", "Region"])
        pd_df = renameChrToRoman(pd_df, "Chromosome")
        palette = {"C": "tab:blue", "G": "tab:red", 'NaN': "tab:gray", "T": "tab:gray", "A": "tab:gray"}
        sns.scatterplot(ax=ax, data=pd_df, x="Region", y="Chromosome", marker="|", s=120, alpha=1, zorder=3, linewidth=1.5, hue="Reference", palette=palette)

    elif map_type=="SNVs_A3A_cluster":
        pd_df = SampleTable_Object.df_SNPs.copy()
        pd_df = pd_df[(((pd_df["Reference"] == "C") & (pd_df["Allele"] == "T")) | ((pd_df["Reference"] == "G") & (pd_df["Allele"] == "A")))]
        pd_df = pd.concat([pd_df, chr_starts_df], ignore_index=True).sort_values(by=["Chromosome", "Region"])
        pd_df = renameChrToRoman(pd_df, "Chromosome")
        palette = {"C": "tab:blue", "G": "tab:red", 'NaN': "tab:gray", "T": "tab:gray", "A": "tab:gray"}
        sns.scatterplot(ax=ax, data=pd_df, x="Region", y="Chromosome", marker="|", s=120, alpha=1, zorder=3, linewidth=1.5, hue="Reference", palette=palette)

        switches_df = SampleTable_Object.switchesTable.copy()
        switches_df = switches_df[switches_df["Chromosome"] != "ref|NC_001224|"]
        switches_df = pd.concat([switches_df, starts_switches], ignore_index=True)
        switches_df = renameChrToRoman(switches_df, "Chromosome")
        sns.scatterplot(ax=ax,data=switches_df, x="Switch_Center", y="Chromosome", color="black", marker=2, s=120, zorder=0, linewidth=2.5, alpha=0.8)

    elif map_type=="SNVs_A3A":
        pd_df = SampleTable_Object.df.copy()
        pd_df = pd_df[(((pd_df["Reference"] == "C") & (pd_df["Allele"] == "T")) | ((pd_df["Reference"] == "G") & (pd_df["Allele"] == "A")))]
        pd_df = pd.concat([pd_df, chr_starts_df], ignore_index=True).sort_values(by=["Chromosome", "Region"])
        pd_df = renameChrToRoman(pd_df, "Chromosome")
        palette = {"C": "tab:blue", "G": "tab:red", 'NaN': "tab:gray", "T": "tab:gray", "A": "tab:gray"}
        sns.scatterplot(ax=ax, data=pd_df, x="Region", y="Chromosome", marker="|", s=120, alpha=1, zorder=3, linewidth=1.5, hue="Reference", palette=palette)

        switches_df = SampleTable_Object.switchesTable.copy()
        switches_df = switches_df[switches_df["Chromosome"] != "ref|NC_001224|"]
        switches_df = pd.concat([switches_df, starts_switches], ignore_index=True)
        switches_df = renameChrToRoman(switches_df, "Chromosome")
        sns.scatterplot(ax=ax,data=switches_df, x="Switch_Center", y="Chromosome", color="black", marker=2, s=120, zorder=0, linewidth=2.5, alpha=0.8)

    elif map_type=="combined":
        import numpy as np
        from natsort import index_natsorted
        from mayo.settings import centromeres as chr_centromeres

        #first deal with the SNPs (swaps of parent 1 and parent 2)
        pd_df = SampleTable_Object.parentalSNPs.copy()
        pd_df_heterozygous = SampleTable_Object.df_SNPs_heterozygous.copy()
        pd_df_heterozygous = pd.concat([pd_df_heterozygous, chr_starts_df], ignore_index=True)
        pd_df_heterozygous = pd_df_heterozygous.sort_values(by="Chromosome", key=lambda x: np.argsort(index_natsorted(pd_df_heterozygous["Chromosome"])))
        pd_df_heterozygous = renameChrToRoman(pd_df_heterozygous, "Chromosome")

        pd_df = pd.concat([pd_df, chr_starts_df], ignore_index=True)
        pd_df = pd_df.sort_values(by="Chromosome", key=lambda x: np.argsort(index_natsorted(pd_df["Chromosome"])))  
        pd_df = renameChrToRoman(pd_df, "Chromosome")

        palette = {"parent_1": sns.color_palette("deep")[2], "parent_2": sns.color_palette("deep")[3]}
        #palette = {"parent_1": "tab:green", "parent_2": "tab:red"}
        markers = {"parent_1": 3, "parent_2": 3}

        sns.scatterplot(ax=ax, data=pd_df_heterozygous, x="Region", y="Chromosome", color="tab:gray", marker=3, s=60, alpha=0.6,zorder=-1, linewidth=0.75)
        sns.scatterplot(ax=ax, data=pd_df, x="Region", y="Chromosome", hue="Parent", palette=palette, style="Parent", markers=markers, s=70, alpha=0.7,zorder=-1, linewidth=0.75)

        #second deal with the SNVs (A3A)
        label_dict = {
            "cen": 'Centromere',
            "SPECTRA_STRANDWISE": "Reference Allele",
            "Position": "Position",
            "G": "G→N",
            "C": "C→N",
            "A": "A→N",
            "T": "T→N",
            "C_to_T": "C→T",
            "C_to_A": "C/G→V/B",
            "C_to_G": "C/G→V/B",

            "G_to_A": "G→A",
            "G_to_C": "C/G→V/B",
            "G_to_T": "C/G→V/B",

            'T_to_A': "A/T→N",
            'T_to_C': "A/T→N",
            'T_to_G': "A/T→N",
            'A_to_C': "A/T→N",
            'A_to_G': "A/T→N",
            'A_to_T': "A/T→N",
            }
        markers = {
            'cen': "o",
            "C_to_T": "$|$",
            "C_to_A": "$|$",
            "C_to_G": "$|$",

            "G_to_A": "$|$",
            "G_to_C": "$|$",
            "G_to_T": "$|$",

            'T_to_A': "$|$",
            'T_to_C': "$|$",
            'T_to_G': "$|$",
            'A_to_C': "$|$",
            'A_to_G': "$|$",
            'A_to_T': "$|$",
            }
        palette = {
            'cen': "white",
            "C_to_T": "tab:red",
            "C_to_A": "tab:olive",
            "C_to_G": "tab:olive",

            "G_to_A": "tab:blue",
            "G_to_C": "tab:olive",
            "G_to_T": "tab:olive",

            'A_to_C': "tab:gray",
            'A_to_G': "tab:gray",
            'A_to_T': "tab:gray",

            'T_to_A': "tab:gray",
            'T_to_C': "tab:gray",
            'T_to_G': "tab:gray"
            }

        df_SNVs = SampleTable_Object.df.copy()

        try:
            df_SNVs["SPECTRA_STRANDWISE"] = df_SNVs.apply(lambda x: findSpectraStrandwise(x["Reference"], x["Allele"]), axis=1)
        except:
            logging.error(f"Error in findSpectraStrandwise function. Skipping sample {SampleTable_Object.name}. The sample dataframe might be empty (size: {len(df_SNVs)}).")
            plt.close()
            return


        #df_SNVs = pd.concat([df_SNVs, chr_starts_df], ignore_index=True).sort_values(by=["Chromosome", "Region"])
        df_SNVs = renameChrToRoman(df_SNVs, "Chromosome")
        chr_centromeres = renameChrToRoman(chr_centromeres, "Chromosome", "numeric")
        df_SNVs = pd.concat([df_SNVs, chr_centromeres], ignore_index=True)
        df_SNVs = df_SNVs.sort_values(by="Chromosome", key=lambda x: np.argsort(index_natsorted(df_SNVs["Chromosome"])))    

        sns.scatterplot(ax=ax, data=df_SNVs[df_SNVs["Reference"].isin(["cen"])], x="Region", y="Chromosome", hue="SPECTRA_STRANDWISE", palette=palette, style="SPECTRA_STRANDWISE", markers=markers, s=100, alpha=1,zorder=2, linewidth=0.75, edgecolor="black")
        sns.scatterplot(ax=ax, data=df_SNVs[~df_SNVs["Reference"].isin(["cen"])], x="Region", y="Chromosome", hue="SPECTRA_STRANDWISE", palette=palette, style="SPECTRA_STRANDWISE", markers=markers, s=100, alpha=0.9,zorder=3, linewidth=0.05, edgecolor="black")
        #sns.scatterplot(ax=ax, data=df_SNVs, x="Region", y="Chromosome", marker="|", s=120, alpha=1, zorder=3, linewidth=1.5, hue="Reference", palette=palette)

        switches_df = SampleTable_Object.switchesTable.copy()
        switches_df = switches_df[switches_df["Chromosome"] != "ref|NC_001224|"]
        switches_df = pd.concat([switches_df, starts_switches], ignore_index=True)
        switches_df = renameChrToRoman(switches_df, "Chromosome")
        sns.scatterplot(ax=ax,data=switches_df, x="Switch_Center", y="Chromosome", color="black", marker=3, s=90, zorder=-2, linewidth=0.75, alpha=0.70)
        
        df_recEvents = RecEventsTable_from_GC_CO_dict(SampleTable_Object.GC_dict, SampleTable_Object.CO_dict)
        #remove heterozygous chromosomes from the recombination events; they are not informative/reliable there
        heterozygous_chr = SampleTable_Object.heterozygous_chromosomes

        if len(df_recEvents) != 0:
            df_recEvents = df_recEvents[~df_recEvents["Chromosome"].isin(heterozygous_chr)]
            df_recEvents = pd.concat([df_recEvents, chr_starts_df], ignore_index=True)
            df_recEvents = renameChrToRoman(df_recEvents, "Chromosome")
            palette_recEvents = {"GC": "tab:blue", "CO": "tab:red"}
            sns.scatterplot(ax=ax, data=df_recEvents, x="Region", y="Chromosome", hue="Type", palette=palette_recEvents, marker=11, s=16, zorder=2, linewidth=0.5, alpha=0.7)
            add_GC_CO_legend_to_ax(ax, palette_recEvents)

        if cluster_kind is None:
            pass

        else:
            #pval_list must be ordered from largest to smallest pvalue to make sense in the legend

            if cluster_kind == "PMACD":
                pval_list = ['p05', 'p01', 'p005', 'p001_def'] #'p0005', #'p0001']
                pval_list.reverse()
            
            elif cluster_kind == "JT":
                pval_list = [0.01, 0.001, 0.0001, 0.00001]
                pval_list.reverse()

            else:
                raise ValueError(f"Cluster kind {cluster_kind} is not supported. Please choose from 'JT' or 'PMACD'.")

            save_name_suffix += f"_{cluster_kind}_clusters"
            add_clusters_patch_to_ax(ax, pval_list, SampleTable_Object, clusters_kind=cluster_kind)

    else: #just clusters
        pd_df = SampleTable_Object.df_SNPs.copy()

        pd_df = pd.concat([pd_df, chr_starts_df], ignore_index=True).sort_values(by=["Chromosome", "Region"])
        pd_df = renameChrToRoman(pd_df, "Chromosome")
        palette = {True: "blue", False: "red"}
        sns.scatterplot(ax=ax, data=pd_df, x="Region", y="Chromosome", marker="|", s=120, alpha=1, zorder=3, linewidth=1.5, hue='Clustered_Proximal', palette=palette) #hue='Switch_Proximal', Is_Clustered, Clustered_Proximal

        switches_df = SampleTable_Object.switchesTable.copy()
        switches_df = switches_df[switches_df["Chromosome"] != "ref|NC_001224|"]
        switches_df = pd.concat([switches_df, starts_switches], ignore_index=True)
        switches_df = renameChrToRoman(switches_df, "Chromosome")
        sns.scatterplot(ax=ax,data=switches_df, x="Switch_Center", y="Chromosome", color="black", marker=2, s=120, zorder=0, linewidth=1.5, alpha=0.8)

    ax.set_xlabel("POSITION")
    ax.set_ylabel("CHROMOSOME")

    if map_type=="combined":

        #dont repeat the same item in the legend
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        #order the legend
        legend_order = ["cen", "Position", "C_to_T", "G_to_A", "C_to_A", "C_to_G", "G_to_C", "G_to_T", "T_to_A", "T_to_C", "T_to_G", "A_to_C", "A_to_G", "A_to_T"]

        #relable the legend texts according to the legend_order

        while True:
            try:
                by_label = {label_dict.get(key): by_label[key] for key in legend_order}
                break

            except Exception as e:
                try:
                    logging.warning(f"Could not relabel legend. {e}. Removing {e} from legend_order and trying again.")
                    #extract the string from the error message
                    e = str(e).split("'")[1]
                    legend_order = [x for x in legend_order if x != str(e)]
                except Exception as e:
                    logging.warning(f"Cound not relabel legend. {e}.")
                    break

        ax.legend(by_label.values(), by_label.keys(), fontsize=10, loc='lower right', bbox_to_anchor=(0.98, 0.05), title=None)
        
        plt.tight_layout()

    else:

        legend_texts = plt.legend().get_texts()
        for i in range(len(legend_texts)):
            label_str = legend_texts[i].get_text()
            #remove NaNs from legend
            if label_str == "NaN":
                legend_texts[i].set_text("")
            if label_str in label_dict:
                new_label = label_dict.get(label_str)

                legend_texts[i].set_text(new_label)
    
    if saveMap:
        if not os.path.exists(f'figures/tests/{map_type}'):
            os.makedirs(f'figures/tests/{map_type}')
        plt.savefig(f'figures/{map_type}/{title}_{map_type}{save_name_suffix}.png', transparent=False, dpi=600)

    if showMap:
        plt.show()

    else:
        # close the plot
        plt.close()

In [ ]:
# (optional) draw a combined map of SNPs, Switches, Mutations, and Clusters (at different p-values) for each sample

for index, i in enumerate(samples_SNVs):

    print(i.heterozygous_chromosomes)

    if i.name == samples_parental_SNPs[index].name:
        print(f"Processing sample {i.name}...")
        i.parentalSNPs = samples_parental_SNPs[index].output_df.copy()
        i.df_SNPs_heterozygous = samples_parental_SNPs[index].df_SNPs_heterozygous.copy()
        
    else:
        print("ERROR: sample names do not match!")
        print(f"{i.name} != {samples_parental_SNPs[index].name}")
        break

    drawSNPMap(i, df_chr, starts, saveMap=True, showMap=True, map_type="combined", save_name_suffix=f"_{SNP_NUM}snp", cluster_kind=CLUSTER_TYPE_ANALYSIS)

    

In [ ]:
save_file_name_clusters_null = f"3x_proximal_{CLUSTER_TYPE_ANALYSIS}_clusters_haploid_setting_{SNP_NUM}snp.csv"


samples_SNVs[:] = [df for df in samples_SNVs if df.name in (haploid_samples_ung1d)]
logging.info("Created a subset of samples for simulation (samples_SNVs)")

switches_list[:] = [df for df in switches_list if df.name in (haploid_samples_ung1d)]
logging.info("Created a subset of samples for simulation (switches_list)")

total_clusters = 0
for i in samples_SNVs:
    i_clusters = i.df_SNPs["Cluster_ID"].nunique()
    total_clusters += i_clusters
    logging.info(f"{i.name} has {i_clusters} clusters")
logging.info(f"######## Total number of clusters in the simulation dataset: {total_clusters} ########")

proximal_clusters = conductNullHypothesisSimulation(switches_list, chromosomes, samples_SNVs, simulation_generations=3, save_file_name=save_file_name_clusters_null)
logging.info(f'Finished writing everything to the file: {save_file_name_clusters_null}')
logging.info('Completed null hypothesis simulation')